In [50]:
import time
import json
import pickle
import numpy as np
import pandas as pd
from classes import *
from datetime import datetime
import plotly.graph_objects as go

In [51]:
DAY = "day_0"
with open(f"data/{DAY}/interp.pkl", "rb") as f:
    sensor_data = pickle.load(f)

print(sensor_data.keys())
print(sensor_data["air"][0].keys())
print(type(sensor_data["air"][0]["t"]))
print(np.array_equal(sensor_data["air"][0]["t"],
                     sensor_data["air"][7]["t"]))
print(np.array_equal(sensor_data["sample"][0]["t"],
                     sensor_data["air"][7]["t"]))

dict_keys(['air', 'sample'])
dict_keys(['t', 'hps'])
<class 'numpy.ndarray'>
True
True


In [52]:
def get_pair_net(sensor_data, sensor_l, sensor_r):
    """ 
    sensor_l: 2, sensor_r: 3
    """
    air_l = np.log(sensor_data["air"][sensor_l]["hps"])
    air_r = np.log(sensor_data["air"][sensor_r]["hps"])
    samp_l = np.log(sensor_data["sample"][sensor_l]["hps"])
    samp_r = np.log(sensor_data["sample"][sensor_r]["hps"])

    diff_l = samp_l - air_l
    diff_r = samp_r - air_r
    net = np.concatenate((diff_l, diff_r), axis=0)
    t = sensor_data["air"][sensor_l]["t"]
    return t, net

In [53]:
def get_daily_net_pair(days, sensor_l, sensor_r):
    ts = None
    nets = None
    for day in days:
        DAY = f"day_{day}"
        with open(f"data/{DAY}/interp.pkl", "rb") as f:
            sensor_data = pickle.load(f)

        t, net = get_pair_net(sensor_data, sensor_l, sensor_r)
        if ts is None:
            ts = t.copy()
            nets = net.copy()
        else:
            ts = np.concatenate((ts, t))
            nets = np.concatenate((nets, net), axis=1)

    return ts, nets

In [61]:
def plot_days_net(t, ys, sl, sr, save=False):
    title = f"Net Sensor {sl}, {sr} Readings"
    fig = go.Figure()

    for y in ys:
        fig.add_trace(go.Scatter(x=t,
                                 y=y,
                                 mode="markers",
                                 showlegend=False))

    fig.update_yaxes(title_text="Resistance (Ohms)")
    fig.update_xaxes(title_text="Timestamp")
    fig.update_layout(height=500, width=1200,
                      title_x=0.5,
                      font_family="Times New Roman",
                      title_font_family="Times New Roman",
                      title=dict(text=title, pad=dict(t=0, r=0, b=0, l=0)),
                      margin=dict(t=50, r=10, b=0, l=0))
    fig.show()
    if save:
        fig.write_image(f"Net_{sl}_{sr}.pdf")

In [62]:
for s in range(4):
    sl, sr = s * 2, s * 2 + 1
    ts, nets = get_daily_net_pair([0, 1, 2, 3], sl, sr)
    plot_days_net(ts, nets, sl, sr, save=True)